In [2]:
#import required libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options

In [3]:
#Adding headless option to avoid browser opening again and again
#options = Options()
#options.headless = True
#options.add_argument('window-size=1366x768')

In [4]:
target_website = 'https://in.indeed.com/'
path=Service('C:/Users/krush/Downloads/chromedriver_win32/chromedriver.exe')
#driver=webdriver.Chrome(service=path,options=options)
driver=webdriver.Chrome(service=path)
driver.get(target_website)
driver.maximize_window()

In [5]:
#locate Search Box
search_box = driver.find_element(By.XPATH,"//div[@id='MosaicProviderRichSearchDaemon']")
input1 = search_box.find_element(By.XPATH,".//input[@id='text-input-what']")
input2 = search_box.find_element(By.XPATH,".//input[@id='text-input-where']")
submit_btn = search_box.find_element(By.XPATH,".//button[@class='yosegi-InlineWhatWhere-primaryButton']")

In [6]:
#pass user input here
role = 'Data Analyst'
input1.send_keys(role)
input2.send_keys('Navi Mumbai')
time.sleep(1)
submit_btn.click()
time.sleep(2)

In [7]:
job_location =[]
company_name = []
job_profile =[]
salary =[]
qualification =[]
job_details_link = []
company_rating =[]

In [8]:
last_page = 5 #mention pages
current_page = 1

while current_page < last_page:

    left_panel = driver.find_element(By.XPATH,"//div[@class='jobsearch-LeftPane']")
    single_job_snippet =  left_panel.find_elements(By.XPATH,".//td[@class='resultContent']")



    for single_job in single_job_snippet:
        job_location.append(single_job.find_element(By.XPATH,".//div[@class='companyLocation']").text)
        job_profile.append(single_job.find_element(By.XPATH,".//h2").text)
        company_name.append(single_job.find_element(By.XPATH,".//span[@class='companyName']").text)
        try:
            company_rating.append(single_job.find_element(By.XPATH,".//a[contains(@class,'ratingLink')]/span").text)
        except:
            company_rating.append("No Rating Found")
        job_details_link.append((single_job.find_element(By.XPATH,".//h2/a")).get_attribute('href'))

        #click on job profile for more info
        single_job.click()
        time.sleep(2)

        #fetch job details from right side
        right_panel = driver.find_element(By.XPATH,"//div[@class='jobsearch-RightPane']")
        try:
            salary.append(right_panel.find_element(By.XPATH,".//div[@id='salaryInfoAndJobType']/span").text)
        except:
            salary.append('Not Mentioned')

    try:
        current_page = current_page + 1 
        nxt_btn = driver.find_element(By.XPATH,'//a[contains(@data-testid,"pagination-page-next")]')
        nxt_btn.click()   
        
    except:
        pass
    
    
driver.quit()

In [9]:
df=pd.DataFrame({'Job Profile':job_profile,'Company Name':company_name,'Company Rating':company_rating,'Job Location':job_location,'Salary':salary,'Job Location':job_location,'Apply for This Job':job_details_link})
df.to_excel('jobs.xlsx')
